# 🎬 SadTalker - AI Talking Avatar (Free)

Tạo video người nói từ ảnh tĩnh + audio

**⚠️ QUAN TRỌNG:**
1. Runtime → Change runtime type → **GPU (T4)**
2. Chạy từng cell từ trên xuống
3. Upload ảnh + audio vào folder bên trái

---

In [ ]:
# 1️⃣ Cài đặt SadTalker (chạy 1 lần, mất 5-10 phút)
!git clone https://github.com/OpenTalker/SadTalker.git
%cd SadTalker
!pip install -r requirements.txt
!bash scripts/download_models.sh
print("✅ Cài đặt xong!")

In [ ]:
# 2️⃣ Upload file (Chạy cell này và upload)
from google.colab import files
import os

print("📤 Upload ảnh người (JPG/PNG):")
uploaded_img = files.upload()
image_path = list(uploaded_img.keys())[0]

print("\n📤 Upload file audio (MP3/WAV):")
uploaded_audio = files.upload()
audio_path = list(uploaded_audio.keys())[0]

print(f"\n✅ Ảnh: {image_path}")
print(f"✅ Audio: {audio_path}")

In [ ]:
# 3️⃣ Tạo video AI Avatar (mất 30-60 giây)
# FIX: Dùng 'crop' và tắt 'gfpgan' để tránh lỗi
!python inference.py \
  --driven_audio {audio_path} \
  --source_image {image_path} \
  --result_dir ./results \
  --still \
  --preprocess crop

print("\n✅ Video đã tạo xong!")
print("📁 Xem trong folder: results/")
print("💡 Note: Không dùng GFPGAN enhancer để tránh lỗi")

In [ ]:
# 4️⃣ Download video về máy
import glob

# Tìm video mới nhất
videos = glob.glob("results/**/*.mp4", recursive=True)
latest_video = max(videos, key=os.path.getctime)

print(f"📥 Downloading: {latest_video}")
files.download(latest_video)
print("✅ Xong! Check folder Downloads của bạn")

---

## 🚀 API Server (Optional - Nâng cao)

Chạy server để project Python gọi vào Colab

In [ ]:
# 🔑 Setup ngrok authtoken (chạy 1 lần đầu tiên)
# Lấy token tại: https://dashboard.ngrok.com/get-started/your-authtoken
!ngrok authtoken 3bA7pxBJB40zByTellNotWQMh9Y_68HyGLzzUxNeLoAzdukth

print("✅ Ngrok token đã được cấu hình!")
print("💡 Bây giờ có thể chạy Cell tiếp theo (API Server)")

In [ ]:
# 5️⃣ (Optional) Tạo API endpoint với Flask + ngrok - IMPROVED VERSION
!pip install flask pyngrok flask-cors

from flask import Flask, request, send_file
from pyngrok import ngrok
import subprocess
import os
import glob

app = Flask(__name__)

@app.route('/health', methods=['GET'])
def health():
    return {'status': 'ok'}, 200

@app.route('/generate', methods=['POST'])
def generate():
    try:
        # Nhận ảnh + audio từ client
        image = request.files.get('image')
        audio = request.files.get('audio')
        
        if not image or not audio:
            return {'error': 'Missing image or audio file'}, 400
        
        # Save files
        os.makedirs('/tmp/avatar_input', exist_ok=True)
        os.makedirs('/tmp/results', exist_ok=True)
        
        image_path = '/tmp/avatar_input/input.jpg'
        audio_path = '/tmp/avatar_input/input.wav'
        image.save(image_path)
        audio.save(audio_path)
        
        # Get optional parameters from client (NEW!)
        preprocess = request.form.get('preprocess', 'crop')  # 'crop' more stable than 'full'
        still_mode = request.form.get('still_mode', 'true') == 'true'
        enhancer = request.form.get('enhancer', 'none')  # 'none' avoids GFPGAN errors
        
        print(f"🎬 Processing with: preprocess={preprocess}, enhancer={enhancer}, still={still_mode}")
        
        # Build command - IMPROVED with safer defaults
        cmd = [
            'python', 'inference.py',
            '--driven_audio', audio_path,
            '--source_image', image_path,
            '--result_dir', '/tmp/results',
            '--preprocess', preprocess  # 'crop' is faster and more stable
        ]
        
        if still_mode:
            cmd.append('--still')
        
        # Only add enhancer if not 'none' (GFPGAN often causes errors)
        if enhancer and enhancer != 'none':
            cmd.extend(['--enhancer', enhancer])
        
        print(f"🎬 Command: {' '.join(cmd)}")
        
        # Run SadTalker with timeout
        result = subprocess.run(
            cmd,
            cwd='/content/SadTalker',
            capture_output=True,
            text=True,
            timeout=120  # 2 minute timeout
        )
        
        if result.returncode != 0:
            error_msg = result.stderr or result.stdout or "Unknown error"
            print(f"❌ SadTalker Error: {error_msg}")
            return {
                'error': f'Inference failed with exit code {result.returncode}',
                'stderr': result.stderr,
                'stdout': result.stdout,
                'command': ' '.join(cmd)
            }, 500
        
        # Find output video
        videos = glob.glob("/tmp/results/**/*.mp4", recursive=True)
        if videos:
            video_path = max(videos, key=os.path.getctime)  # Latest video
            print(f"✅ Video generated: {video_path}")
            return send_file(video_path, mimetype='video/mp4')
        else:
            return {'error': 'No video output generated'}, 500
            
    except subprocess.TimeoutExpired:
        print("❌ Timeout: Processing took more than 2 minutes")
        return {'error': 'Processing timeout (>2 minutes)'}, 500
    except Exception as e:
        import traceback
        error_detail = traceback.format_exc()
        print(f"❌ Exception: {error_detail}")
        return {'error': str(e), 'detail': error_detail}, 500

# Khởi động ngrok tunnel
public_url = ngrok.connect(5000)
print(f"\n🌐 API URL: {public_url}")
print("\n📋 COPY URL này vào project Python của bạn!")
print("💡 Set environment variable (in PowerShell):")
print(f"   $env:COLAB_API_URL='{public_url}'")
print("\n⚠️ Giữ cell này chạy (không stop)")
print("💡 Improvements: Dùng 'crop' + 'none enhancer' cho stability cao\n")

# Start Flask server
app.run(port=5000)